# XGBoost Model

## Method explanation

The `xgboost` module is a popular implementation of tree boosting. One of its main differences with the regular boosting introduced in class is the presence of a regularization term directly in the objective function to minimize. The following discussion is based on [XGBoost's documentation](https://xgboost.readthedocs.io/en/stable/index.html) and [Chen, T., & Guestrin, C. (2016, August). Xgboost: A scalable tree boosting system. In Proceedings of the 22nd acm sigkdd international conference on knowledge discovery and data mining (pp. 785-794).](https://arxiv.org/pdf/1603.02754.pdf)

We recall that as we have seen the final boosted model is a sum of $B$ trees which can be written as

$$\hat{f}(x) = \sum_{b=1}^B \hat{f}^b(x)$$

In classical boosting, during training the model is minimizing

$$\mathcal{L}(\hat{f}) = \sum_{i=1}^n \eta L(\hat{y}_i, y_i) \quad \quad \text{where } \hat{y}_i = \hat{f}(x_i)$$

Here, $\eta \in (0,1)$ is the shrinkage parameter and $L$ is the loss function. For regression, this is typically the squared error loss, whereas for classification, this could be the maximum likelihood loss. XGBoost, on the other hand, introduces a second term in the objective to minimize, which can then be rewritten as

$$\mathcal{L}(\hat{f}) = \sum_{i=1}^n \eta L(\hat{y}_i, y_i) + \sum_{b=1}^B \Omega(\hat{f}^b) \quad \quad
\text{where }\Omega(f) = \gamma T + \dfrac{1}{2} \lambda ||w||^2$$

The parameters used in the regularization function $\Omega$ are listed below:
- $T$ is the number of leaves of the tree. This penalizes bigger leaf numbers, which means trees which are more complex.
- $w \in \mathbb{R}^T$ are the weights which are assigned in each leaf of the tree. The weights are the predicted outputs in each leaf. They are real values if we have a regression tree, and probabilities if we have a decision tree.
- $\gamma \geq 0$ and $\lambda \geq 0$ are penalty terms, akin to parameter $\lambda$ in ridge or lasso linear regression, for example. The bigger their values, the stronger the regularization is. They are tuning parameters.


The implementation of regularization directly in the objective function helps to avoid overfitting which can happen easily when using boosting methods. XGBoost also provides a lot of other improvements from a point of view of computational efficiency. We will not get in the details as their are more directly related to computer science, however some aspects will be alluded to later when discussing all the tuning parameters we have used to build our model.

## Preliminary steps

First, we import the necessary modules.

In [625]:
import pandas as pd
import numpy as np
from scipy import stats
import csv

from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score

from xgboost import XGBClassifier

We import the data and split it into predictors `X` and response `y`.

In [626]:
campaign_ad = pd.read_csv("MLUnige2023_subscriptions_train.csv", index_col="Id")
campaign_test = pd.read_csv("MLUnige2023_subscriptions_test.csv", index_col="Id")

X = campaign_ad.drop(columns='subscription')
y = campaign_ad['subscription']

XGBoost can handle categorical variables, but their `dtype` must be converted to `category`. We do so below.

In [627]:
# numerical variables
num_vars = ['age', 'time_spent', 'banner_views', 'banner_views_old', 'days_elapsed_old', 'X4']
# categorical variables
cat_vars = list(set(X.columns).difference(num_vars))

# converting type to category
for col in cat_vars:
  X[col] = X[col].astype("category")
  campaign_test[col] = campaign_test[col].astype("category")

We split `X` and `y` into training, validation and test sets. The seeds are indentical to the ones used with other models to ensure that we have a fair ground.

In [628]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=12)
X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=46)

We define our `xgb_clf` below with the following parameters:
- `tree_method` is set to `'hist'`. One of the particularities implemented by XGBoost is that when choosing the next best split for a tree, instead of sorting all values of each predictor and go through them to determine where the optimal split should occur, the algorithm will create an histogram which approximates the distribution of the predictor. This makes the whole process much more fast.
- `objective` is set to `'binary:logistic'`. This is the appropriate objective function when dealing with a classification task in which the outome variable is binary.
- `eval_metric` is set to `'auc'`. In classification, a commonly used visualization to assess model performance is the ROC curve, which plots the True Positive Rate against the False Positive Rate. A model which performs well should have a curve which increases as steeply and quickly as possible. Thus, a possible measure used to assess the model performance is the area under the ROC curve, which is called AUC. A bigger area means that the model is performing better. The maximum area of the ROC curve is 1. The AUC is handy, as it does not depend on the choice of probablility cutoff for classifying records as 0 or 1.
- `enable_categorical` is set to `True`. This is necessary if there are categorical predictors, so that XGBoost correctly handles them.
- `n_jobs` is set to `-2`. This parameter enambles portions of the algorithm to be run in parallel on multiple cores of the CPU.

In [629]:
xgb_clf = XGBClassifier(tree_method='hist',
                        objective='binary:logistic',
                        eval_metric='auc',
                        enable_categorical=True,
                        n_jobs=-2,
                        random_state=391)

Now, we define a parameter grid for our parameter search. We explain the tuning parameters which are used below:
- `n_estimators` is the number of trees $B$.
- `max_depth` is the maximum depth allowed for all trees.
- `learning_rate` is the shrinkage parameter $\eta$.
- `colsample_bytree` is the proportion of the total features which are used to grow each tree. For example, if it is equal to 0.5, it means that for each tree, only half the predictors are randomly picked among all possible features. Thus, each tree has its own combination of predictors. By default, it is set to 1.
- `gamma` is the penalty parameter $\gamma$ defined inside regularization function $\Omega$.
- `reg_lambda` is the penalty parameter $\lambda$ defined inside regularization function $\Omega$.

In [630]:
param_search = {'n_estimators': range(50, 301),
                'max_depth': range(1, 8),
                'learning_rate': stats.uniform(0.001, 0.3),   # [0.001, 0.301]
                'colsample_bytree': stats.uniform(0.5, 0.5),  # [0.5, 1]
                'gamma': stats.uniform(0, 0.5),               # [0, 0.5]
                'reg_lambda': stats.uniform(0, 5)             # [0, 5]
                }

In the above grid, for continuous tuning parameters, we used a uniform distribution inside an interval instead of a list of values. This is because we are going to use `RandomizedSearchCV` instead of the classical grid search. This gridsearch will generate random values from the distributions we provided. To do so, in each fold it will go through a number `n_iter` of iterations taken from the grid we constructed. We have chosen to use the `StratifiedKFold` function which creates folds which all have a similar proportion of positive responses.

In [631]:
model = RandomizedSearchCV(xgb_clf,
                           param_distributions=param_search,
                           n_iter=250,
                           scoring='roc_auc',
                           verbose=1,
                           cv= StratifiedKFold(n_splits=5, shuffle=True, random_state=611),
                           random_state=3989)

model.fit(X_train, y_train)

Fitting 5 folds for each of 250 candidates, totalling 1250 fits


RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=611, shuffle=True),
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=True,
                                           eval_metric='auc',
                                           feature_types=None, gamma=None,
                                           gpu_id=None, grow_policy=None,
                                           importa...
                                        'gamma': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7fd8a89709a0>,
                                        'learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7fd8a8916df0>,
                                        'max_depth': range(1, 8),
                                        'n_estimators': range(50, 301),
                                        'reg_lambda': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7fd8aa9cbdf0>},
                   random_state=3989, scoring='roc_auc', verbose=1)

We can now retrieve the best parameters from our search.

In [682]:
best_params = model.best_params_
best_params

{'colsample_bytree': 0.5771695955273806,
 'gamma': 0.004545655718141806,
 'learning_rate': 0.06939821015677476,
 'max_depth': 7,
 'n_estimators': 270,
 'reg_lambda': 1.0061012533676594}

We can also look at the AUC of the best model.

In [683]:
model.best_score_

0.9308282426468221

This is a good score, as the maximal value of the AUC is 1. Now, we can use the best parameters to fit our XGBoost model.

In [695]:
xgb_model = XGBClassifier(**best_params,
                          tree_method='hist',
                          objective='binary:logistic',
                          eval_metric='auc',
                          enable_categorical=True)

xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.5771695955273806, early_stopping_rounds=None,
              enable_categorical=True, eval_metric='auc', feature_types=None,
              gamma=0.004545655718141806, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.06939821015677476, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=7, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=270, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

We compute the accuracy scores on the training, validation and test sets of the model. We also compute it on `campaign_ad`as a whole.

In [703]:
y_train_pred = xgb_model.predict(X_train)
print('Accuracy on train:', accuracy_score(y_train, y_train_pred))

y_valid_pred = xgb_model.predict(X_valid)
print('Accuracy on valid:', accuracy_score(y_valid, y_valid_pred))

y_test_pred = xgb_model.predict(X_test)
print('Accuracy on test:', accuracy_score(y_test, y_test_pred))

y_pred = xgb_model.predict(X)
print('Accuracy on campaign_ad:', accuracy_score(y, y_pred))

Accuracy on train: 0.984360038301947
Accuracy on valid: 0.9888309754281459
Accuracy on test: 0.9843633655994043
Accuracy on campaign_ad: 0.9850312779267203


Now, we fit the model on the whole `campaign_ad` dataset before generating the prediction for `campaign_test`. We also display the accuracy of the model on `campaign_ad` once it is fitted.

In [705]:
xgb_model.fit(X, y)
y_pred = xgb_model.predict(X)
print('Accuracy on campaign_ad for model trained on campaign_ad:', accuracy_score(y, y_pred))

y_REAL_test = xgb_model.predict(campaign_test)

Accuracy on campaign_ad for model trained on campaign_ad: 0.9850312779267203


Finally, we write the computed predictions `y_REAL_test` into a CSV file which is then uploaded onto kaggle.

In [692]:
file = open('test_file_xgboost.csv', 'w')
writer = csv.writer(file)
writer.writerow(['Id', 'subscription'])
for i in range(len(y_REAL_test)):
    writer.writerow([i, y_REAL_test[i]])
file.close()